In [19]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: ai-ml-udacity
Azure region: centralus
Subscription id: 5cc9c356-8bdb-4c04-ae9b-a7378188e1e3
Resource group: bharat-udacity-learn


In [20]:
from azureml.core.compute import ComputeTarget, AmlCompute

# Cluster name 
cluster_name = "Bharat-udacity-project"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Configure the cluster
compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)

# Create the compute cluster
try:
  # Check if cluster already exists (optional)
  cluster = ComputeTarget(workspace=ws, name=cluster_name)
  print(f"Found existing cluster: {cluster_name}")
except:
  # If not found, create a new cluster
  cluster = ComputeTarget.create(workspace=ws, name=cluster_name, compute_config=compute_config)
  cluster.wait_for_completion(show_output=True)  # Wait for cluster creation

print(f"Cluster {cluster_name} created successfully!")

Found existing cluster: Bharat-udacity-project
Cluster Bharat-udacity-project created successfully!


In [21]:
pip install azureml-sdk[automl,notebooks,explain,train]

Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install azureml-sdk[widgets]

Note: you may need to restart the kernel to use updated packages.


In [24]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
# Specify parameter sampler
src_space = {
    "learning_rate": uniform(min_value=0.001, max_value=0.1),
    "batch_size": choice(32, 64, 128)
}
ps = RandomParameterSampling({'C':uniform(0.5,2.0),
                             'max_iter': choice(50, 100, 150, 200)})

# Specify a Policy
# Example with both slack_factor and evaluation_interval
policy = BanditPolicy(slack_factor=0.2, evaluation_interval=50)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(
    source_directory='.',  # Path to your training script directory
    script='train.py',                 # Name of your training script
    compute_target=cluster,  # Name of the compute target for training
    environment=sklearn_env      # Name of the environment to use for training
)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
# Define HyperDriveConfig object
hyperdrive_config = HyperDriveConfig(
    run_config=src,  # Use the defined ScriptRunConfig object
    primary_metric_name="Accuracy",  # Set the primary metric for evaluation
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,  # Set the goal (maximize/minimize)
    max_concurrent_runs=4,
    max_total_runs=20,  # Set the maximum number of runs
    hyperparameter_sampling=ps,
    policy=policy  # Include policy object if using early termination
)

In [25]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
run = exp.submit(config=hyperdrive_config)
run.wait_for_completion(show_output=True)

RunId: HD_b26b38e9-8448-4e41-91fe-ba615b5be6fc
Web View: https://ml.azure.com/runs/HD_b26b38e9-8448-4e41-91fe-ba615b5be6fc?wsid=/subscriptions/5cc9c356-8bdb-4c04-ae9b-a7378188e1e3/resourcegroups/bharat-udacity-learn/workspaces/ai-ml-udacity&tid=db05faca-c82a-4b9d-b9c5-0f64b6755421

Streaming azureml-logs/hyperdrive.txt

[2024-05-19T11:12:21.401309][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2024-05-19T11:12:21.9040232Z][SCHEDULER][INFO]Scheduling job, id='HD_b26b38e9-8448-4e41-91fe-ba615b5be6fc_0' 
[2024-05-19T11:12:22.0359723Z][SCHEDULER][INFO]Scheduling job, id='HD_b26b38e9-8448-4e41-91fe-ba615b5be6fc_1' 
[2024-05-19T11:12:22.1732767Z][SCHEDULER][INFO]Scheduling job, id='HD_b26b38e9-8448-4e41-91fe-ba615b5be6fc_2' 
[2024-05-19T11:12:22.2972587Z][SCHEDULER][INFO]Scheduling job, id='HD_b26b38e9-8448-4e41-91fe-ba615b5be6fc_3' 
[2024-05-19T11:12:22.263662][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[202

{'runId': 'HD_b26b38e9-8448-4e41-91fe-ba615b5be6fc',
 'target': 'bharat-udacity-project',
 'status': 'Completed',
 'startTimeUtc': '2024-05-19T11:11:50.786245Z',
 'endTimeUtc': '2024-05-19T11:23:58.308688Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'e5506ce7-fb62-4732-adf3-74dd9a37ad4d',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1040-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.51.0',
  'space_size': 'infinite_space_size',
  'best_child_run_id': 'HD_b26b38e9-8448-4e41-91fe-ba615b5be6fc_7',
  'score': '0.9182094081942336',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_b26b38e9-8448-4e41-91fe-ba615b5be6fc_7'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'te

In [26]:
RunDetails(run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [33]:
best_run = run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()

print(best_run)
print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
# print('\n learning rate:',parameter_values[3])
# print('\n keep probability:',parameter_values[5])
# print('\n batch size:',parameter_values[7])

Run(Experiment: udacity-project,
Id: HD_b26b38e9-8448-4e41-91fe-ba615b5be6fc_7,
Type: azureml.scriptrun,
Status: Completed)
Best Run Id:  HD_b26b38e9-8448-4e41-91fe-ba615b5be6fc_7

 Accuracy: 0.9182094081942337


In [34]:
best_run.get_file_names()

['logs/azureml/dataprep/0/backgroundProcess.log',
 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log',
 'logs/azureml/dataprep/0/rslex.log.2024-05-19-11',
 'outputs/model.pkl',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/metrics_capability/metrics-capability.log',
 'system_logs/snapshot_capability/snapshot-capability.log',
 'user_logs/std_log.txt']

In [35]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run.register_model(model_name="hdr_config_best_model", model_path=os.path.join("outputs", "model.pkl"))

Model(workspace=Workspace.create(name='ai-ml-udacity', subscription_id='5cc9c356-8bdb-4c04-ae9b-a7378188e1e3', resource_group='bharat-udacity-learn'), name=hdr_config_best_model, id=hdr_config_best_model:1, version=1, tags={}, properties={})

In [36]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

file_url_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
tabular_dataset = TabularDatasetFactory.from_delimited_files(path=file_url_path)

In [37]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(tabular_dataset)

In [38]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=tabular_dataset,
    label_column_name='y',
    compute_target=cluster,
    n_cross_validations=5,
    enable_onnx_compatible_models=True)

In [39]:
# Submit your automl run

run = exp.submit(automl_config)
RunDetails(run).show()
run.wait_for_completion(show_output=True)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_6ad66fb8-1c11-49cb-8885-2a9f859199c7,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_6ad66fb8-1c11-49cb-8885-2a9f859199c7,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+-------------------------------------

{'runId': 'AutoML_6ad66fb8-1c11-49cb-8885-2a9f859199c7',
 'target': 'bharat-udacity-project',
 'status': 'Completed',
 'startTimeUtc': '2024-05-19T11:35:21.678487Z',
 'endTimeUtc': '2024-05-19T12:15:00.283411Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 30 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'bharat-udacity-project',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"5cc9c356-8bdb-4c04-ae9b-a7378188e1e3","resource_group":"bharat-udacity-learn","workspace_name":"ai-ml-udacity","region":"centralus","compute_target":"bharat-udacity-project","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_ma

In [ ]:
# Retrieve and save your best automl model.
import joblib

best_run, fit_model = run.get_output()
print(best_run)

os.mkdir("model_output")
os.chdir("model_output")
joblib.dump(fit_model,"AUTOML.pkl")